In [34]:
import numpy as np
import pandas as pd

In [35]:
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score

from sklearn.compose import ColumnTransformer

In [36]:
df = pd.read_csv('titanic.csv')[['Age','Fare','SibSp','Parch','Survived']]

In [37]:
df.dropna(inplace=True)

In [38]:
df.head()

,Age,Fare,SibSp,Parch,Survived
0,22.0,7.2500,1,0,0
1,38.0,71.2833,1,0,1
2,26.0,7.9250,0,0,1
3,35.0,53.1000,1,0,1
4,35.0,8.0500,0,0,0


In [39]:
df['family'] = df['SibSp'] + df['Parch']

In [40]:
df.head()

,Age,Fare,SibSp,Parch,Survived,family
0,22.0,7.2500,1,0,0,1
1,38.0,71.2833,1,0,1,1
2,26.0,7.9250,0,0,1,0
3,35.0,53.1000,1,0,1,1
4,35.0,8.0500,0,0,0,0


In [41]:
df.drop(columns=['SibSp','Parch'],inplace=True)

In [42]:
df.head()

,Age,Fare,Survived,family
0,22.0,7.2500,0,1
1,38.0,71.2833,1,1
2,26.0,7.9250,1,0
3,35.0,53.1000,1,1
4,35.0,8.0500,0,0


In [43]:
df.sample(100)

,Age,Fare,Survived,family
841,16.0,10.5000,0,0
870,26.0,7.8958,0,0
597,49.0,0.0000,0,0
604,35.0,26.5500,1,0
509,26.0,56.4958,1,0
...,...,...,...,...
50,7.0,39.6875,0,5
118,24.0,247.5208,0,1
599,49.0,56.9292,1,1
216,27.0,7.9250,1,0



# train test split

In [44]:
X = df.drop(columns=['Survived'])
y = df['Survived']

In [45]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [46]:
X_train.head()

,Age,Fare,family
328,31.0,20.5250,2
73,26.0,14.4542,1
253,30.0,16.1000,1
719,33.0,7.7750,0
666,25.0,13.0000,0


In [47]:
# Without binarization

clf = DecisionTreeClassifier()

clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

accuracy_score(y_test,y_pred)*100

62.23776223776224

In [48]:
np.mean(cross_val_score(DecisionTreeClassifier(),X,y,cv=10,scoring='accuracy'))*100

64.85328638497651

# Applying binarization

In [59]:
# Applying Binarization

from sklearn.preprocessing import Binarizer

In [60]:
trf = ColumnTransformer([
    ('bin',Binarizer(threshold=0,copy=False),['family'])
],remainder='passthrough')

In [53]:
X_train_trf = trf.fit_transform(X_train)
X_test_trf = trf.transform(X_test)

In [54]:
pd.DataFrame(X_train_trf,columns=['family','Age','Fare'])

,family,Age,Fare
0,1.0,31.0,20.5250
1,1.0,26.0,14.4542
2,1.0,30.0,16.1000
3,0.0,33.0,7.7750
4,0.0,25.0,13.0000
...,...,...,...
566,1.0,46.0,61.1750
567,0.0,25.0,13.0000
568,0.0,41.0,134.5000
569,1.0,33.0,20.5250


In [55]:
clf = DecisionTreeClassifier()
clf.fit(X_train_trf,y_train)
y_pred2 = clf.predict(X_test_trf)

accuracy_score(y_test,y_pred2)*100

61.53846153846154

In [56]:
X_trf = trf.fit_transform(X)
np.mean(cross_val_score(DecisionTreeClassifier(),X_trf,y,cv=10,scoring='accuracy'))*100

63.32159624413146